# Doctor Who Data Aggregation

This notebook seeks to merge various Doctor Who data sources together into a single `Merged.csv` file of further analysis.

In [56]:
# Imports
import pandas as pd
import numpy
import datetime

In [57]:
# Load Episodes
df_eps = pd.read_csv('../Data/all-detailsepisodes.csv')

# First Diffusion seems to indicate the air date so let's rename it for clarity
df_eps.rename(columns={'first_diffusion': 'air_date'}, inplace=True)

# We've got some messed up data in this dataset so let's do some cleansing
df_eps['air_date'] = numpy.where(df_eps.air_date == 'Death in Heaven', '8-Nov-14', df_eps.air_date)
df_eps['air_date'] = numpy.where(df_eps.air_date == 'Into the Dalek', '30-Aug-14', df_eps.air_date)
df_eps['air_date'] = numpy.where(df_eps.air_date == 'The ZygonInvasion', '31-Oct-15', df_eps.air_date)
df_eps['air_date'] = numpy.where(df_eps.air_date == 'Time Heist', '20-Sep-14', df_eps.air_date)

# Correct some titles. At the end we will be merging by title so title consistency is important
df_eps['title'] = numpy.where(df_eps.title == 'A Good Man Goes To War', 'A Good Man Goes to War', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'Daleks In Manhattan', 'Daleks in Manhattan', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'Evolution Of The Daleks', 'Evolution of the Daleks', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'Face the Raven', 'Face The Raven', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'Journey to the Centre of the Tardis', 'Journey to the Centre of the TARDIS', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'The Unicorn and The Wasp', 'The Unicorn and the Wasp', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'The Woman Who Fell To Earth', 'The Woman Who Fell to Earth', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'The Return of Doctor Mysterio, by Stephen Moffat', 'The Return of Doctor Mysterio', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'The Day of the Doctor', 'The Day of The Doctor', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'The End of Time, part one', 'The End of Time: Part One', df_eps.title)
df_eps['title'] = numpy.where(df_eps.title == 'The End of Time, part two', 'The End of Time: Part Two', df_eps.title)

# Filter down to "New Who"
df_eps = df_eps[df_eps['doctorid'] >= 9]

df_eps = df_eps.sort_values('air_date')

# Preview
df_eps#.head()

,episodeid,title,air_date,doctorid
179,36-8,The Lie of the Land,03 Jun 2017,12
208,37-1,The Woman Who Fell to Earth,07 Oct 2018,13
218,37-11,Resolution,1 Jan 2019,13
120,30-18,The End of Time: Part Two,"1 Jan, 2010",10
183,36-12,The Doctor Falls,1 Jul 2017,12
...,...,...,...,...
156,34-12,Death in Heaven,8-Nov-14,12
317,27-3,The Unquiet Dead,"9 Apr, 2005",9
217,37-10,The Battle of Ranskoor Av Kolos,9 Dec 2018,13
96,29-10,Blink,"9 Jun, 2007",10


In [58]:
# Load IMDB Details
df_imdb = pd.read_csv('../Data/imdb_details.csv')

# Correct the season to be consistent with the eps dataset
df_imdb['season'] = df_imdb['season'] + 26

# Generate a consistent ID we can use for joining / merging
df_imdb['episode_id'] = df_imdb['season'].astype(str) + '-' + df_imdb['number'].astype(str)

# Simplify column
df_imdb.rename(columns={'nbr_votes': 'votes'}, inplace=True)

# Preview
df_imdb

,number,title,rating,votes,description,season,episode_id
0,1,Rose,7.6,6504,When ordinary shop-worker Rose Tyler meets a m...,27,27-1
1,2,The End of the World,7.6,5684,The Doctor takes Rose to the year 5 billion to...,27,27-2
2,3,The Unquiet Dead,7.6,5326,The Doctor has great expectations for his late...,27,27-3
3,4,Aliens of London,7.0,5116,The Doctor returns Rose to her own time - well...,27,27-4
4,5,World War Three,7.1,4943,The Slitheen have infiltrated Parliament and h...,27,27-5
...,...,...,...,...,...,...,...
143,7,Kerblam!,6.4,3954,A mysterious message arrives in a package addr...,37,37-7
144,8,The Witchfinders,5.9,3635,"Arriving in 17th Century Lancashire, the TARDI...",37,37-8
145,9,It Takes You Away,6.4,3404,"On the edge of a Norwegian fjord, in the prese...",37,37-9
146,10,The Battle of Ranskoor Av Kolos,5.5,2949,"Answering nine separate distress calls, the Do...",37,37-10


In [59]:
# Join the IMDB and episodes datasets together on their episode ids

# Remove the title column from the right table for a cleaner merge
df_right = df_imdb.drop(columns=['title'])

# Merge the datasets into a new dataset
df_merged = pd.merge(df_eps, df_right, left_on='episodeid', right_on='episode_id', how='inner')

# Clean up the merged dataset
df_merged.drop(columns=['episodeid'], inplace=True)
df_merged

,title,air_date,doctorid,number,rating,votes,description,season,episode_id
0,The Lie of the Land,03 Jun 2017,12,8,7.2,2560,Earth is invaded and Bill is living alone in o...,36,36-8
1,The Woman Who Fell to Earth,07 Oct 2018,13,1,7.1,8066,"In a South Yorkshire city, Ryan Sinclair, Yasm...",37,37-1
2,Resolution,1 Jan 2019,13,11,6.0,2690,"As the New Year begins, a terrifying evil is s...",37,37-11
3,The Doctor Falls,1 Jul 2017,12,12,9.0,3584,The Doctor makes a final stand against an army...,36,36-12
4,Army of Ghosts,"1 Jul, 2006",10,12,8.5,4885,When ghosts of loved ones appear all over the ...,28,28-12
...,...,...,...,...,...,...,...,...,...
141,Dinosaurs on a Spaceship,"8 Sep, 2012",11,2,7.5,4676,"After a lengthy absence, the Doctor returns to...",33,33-2
142,Death in Heaven,8-Nov-14,12,12,7.9,4340,"With Cybermen on the streets of London, old fr...",34,34-12
143,The Unquiet Dead,"9 Apr, 2005",9,3,7.6,5326,The Doctor has great expectations for his late...,27,27-3
144,The Battle of Ranskoor Av Kolos,9 Dec 2018,13,10,5.5,2949,"Answering nine separate distress calls, the Do...",37,37-10


In [60]:
# Load Doctor Who Guide
df_guide = pd.read_csv('../Data/dwguide.csv')

# Column notes:
# - episodenbr may correlate to episodeid in df_eps, but the name would need to be standardized
# - views needs cleaning to be numeric
# - share seems to have missing values
# - AI is unclear in meaning
# - chart is unclear in meaning
# - Cast is a JSON array
# - Crew is a JSON array
# - Summary has missing values

# Interpret our dates correctly and filter down to "New Who"
df_guide['date'] = pd.to_datetime(df_guide['broadcastdate'])
df_guide.drop(columns=['broadcastdate'], inplace=True)
df_guide = df_guide[df_guide['date'] > datetime.datetime(2000,1,1)]

# Fix titles so we can join appropriately
df_guide['title'] = numpy.where(df_guide.title == 'Arachnids In The UK', 'Arachnids in the UK', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'Empress Of Mars', 'Empress of Mars', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'Demons Of The Punjab', 'Demons of the Punjab', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'In The Forest Of The Night', 'In the Forest of the Night', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'Kill The Moon', 'Kill the Moon', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'Love & Monsters', 'Love and Monsters', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'Mummy On The Orient Express', 'Mummy on the Orient Express', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'Robot Of Sherwood', 'Robot of Sherwood', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'The End Of The World', 'The End of the World', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'The Power Of Three', 'The Power of Three', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'The Pyramid At The End Of The World', 'The Pyramid at the End of the World', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'The Return Of Doctor Mysterio', 'The Return of Doctor Mysterio', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'World Enough And Time', 'World Enough and Time', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'The Curse of the Black Spot', 'Curse of the Black Spot', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'The Doctor, The Widow and the Wardrobe', 'The Doctor, the Widow, and the Wardrobe', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'The Family of Blood', 'Family of Blood', df_guide.title)
df_guide['title'] = numpy.where(df_guide.title == 'The Lie Of The Land', 'The Lie of the Land', df_guide.title)

# Preview
df_guide

,episodenbr,title,weekday,broadcasthour,duration,views,share,AI,chart,cast,crew,summary,date
96,697,Rose,Sat,7:00pm,00:44:14,10.81m,44.8%,76.0,7,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Russell T Davies""},{...",NaN,2005-03-26
97,698,The End of the World,Sat,6:59pm,00:44:45,7.97m,37.8%,76.0,19,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Russell T Davies""},{...",NaN,2005-04-02
98,699,The Unquiet Dead,Sat,7:00pm,00:44:50,8.86m,37.8%,80.0,15,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Mark Gatiss""},{""role...",NaN,2005-04-09
99,700,Aliens of London,Sat,7:00pm,00:45:05,7.63m,35.7%,82.0,18,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Russell T Davies""},{...",NaN,2005-04-16
600,701,World War Three,Sat,7:01pm,00:40:40,7.98m,40.2%,81.0,20,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Russell T Davies""},{...",NaN,2005-04-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,847,Kerblam!,Sun,6:30pm,00:48:50,7.46m,31.8%,81.0,9,"[{""role"":""The Doctor"",""name"":""Jodie Whittaker""...","[{""role"":""Writer"",""name"":""Pete McTighe""},{""rol...",“Delivery for the Doctor!” \n\n \n\nA mysterio...,2018-11-18
847,848,The Witchfinders,Sun,6:30pm,00:46:31,7.21m,31.5%,81.0,17,"[{""role"":""The Doctor"",""name"":""Jodie Whittaker""...","[{""role"":""Writer"",""name"":""Joy Wilkinson""},{""ro...","The Doctor, Ryan, Graham and Yaz arrive in 17t...",2018-11-25
848,849,It Takes You Away,Sun,6:31pm,00:49:11,6.42m,28.3%,80.0,22,"[{""role"":""The Doctor"",""name"":""Jodie Whittaker""...","[{""role"":""Writer"",""name"":""Ed Hime""},{""role"":""D...","On the edge of a Norwegian fjord, in the prese...",2018-12-02
849,850,The Battle of Ranskoor Av Kolos,Sun,6:25pm,00:49:40,6.65m,29.6%,79.0,18,"[{""role"":""The Doctor"",""name"":""Jodie Whittaker""...","[{""role"":""Writer"",""name"":""Chris Chibnall""},{""r...","On the planet of Ranskoor Av Kolos, lies the r...",2018-12-09


In [61]:
# There's not a good way to translate the episode number to a season and episode number for joining, so let's try joining between our merged table and the Doctor Who guide table by title
df_trio = pd.merge(df_guide, df_merged, left_on='title', right_on='title', how='inner')

# Translate the season number to be relative to "New Who"
df_trio['season'] = df_trio['season'] - 26
df_trio['episode_id'] = df_trio['season'].astype(str) + '.' + df_trio['number'].astype(str).str.rjust(2, "0")
df_trio.set_index('episode_id', inplace=True)

df_trio.head()

,episodenbr,title,weekday,broadcasthour,duration,views,share,AI,chart,cast,crew,summary,date,air_date,doctorid,number,rating,votes,description,season
episode_id,,,,,,,,,,,,,,,,,,,,
1.01,697,Rose,Sat,7:00pm,00:44:14,10.81m,44.8%,76.0,7,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Russell T Davies""},{...",NaN,2005-03-26,"26 Mar, 2005",9,1,7.6,6504,When ordinary shop-worker Rose Tyler meets a m...,1
1.02,698,The End of the World,Sat,6:59pm,00:44:45,7.97m,37.8%,76.0,19,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Russell T Davies""},{...",NaN,2005-04-02,"2 Apr, 2005",9,2,7.6,5684,The Doctor takes Rose to the year 5 billion to...,1
1.03,699,The Unquiet Dead,Sat,7:00pm,00:44:50,8.86m,37.8%,80.0,15,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Mark Gatiss""},{""role...",NaN,2005-04-09,"9 Apr, 2005",9,3,7.6,5326,The Doctor has great expectations for his late...,1
1.04,700,Aliens of London,Sat,7:00pm,00:45:05,7.63m,35.7%,82.0,18,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Russell T Davies""},{...",NaN,2005-04-16,"16 Apr, 2005",9,4,7.0,5116,The Doctor returns Rose to her own time - well...,1
1.05,701,World War Three,Sat,7:01pm,00:40:40,7.98m,40.2%,81.0,20,"[{""role"":""Doctor Who"",""name"":""Christopher Eccl...","[{""role"":""Writer"",""name"":""Russell T Davies""},{...",NaN,2005-04-23,"23 Apr, 2005",9,5,7.1,4943,The Slitheen have infiltrated Parliament and h...,1


In [62]:
# Save it as a CSV for future analysis
df_trio.to_csv('../Data/Merged.csv')